In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()
import sys
import os
import base64
from PIL import Image
from io import BytesIO
sys.path.insert(0, os.path.abspath('..'))

In [2]:
import time, base64, json, requests, asyncio
import nest_asyncio
nest_asyncio.apply()
from typing import List, Dict, Union, Any 
import logging
# logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', force=True)     
# Import our providers
from llm_master import QueryLLM, LLMConfig
config = LLMConfig.from_env()
llm = QueryLLM(config)



/Users/sina/Desktop/Uflo Platform/uflo-AI-server/ai-server/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:llm_master.response_synthesizer:Initialized QueryLLM handler with rate limiters


In [3]:
 
async def run_code_interpreter():

    try:
        # With stream=False, you get a LLMResponse object directly
        response_generator = await llm.query(
            model_name="responses-o4-mini",
            messages=[],
            stream=True,
            tools=[{
                "type": "code_interpreter",
                "container": {
                    "type": "auto",
                    # "file_ids": ["file-TXT3RH5yycr7MAX2H8kLvq"]
                }
            }],
            input=[{
                "role": "user",
                "content": [{ "type": "input_text", "text": "Can you make a simple word document for lesson plan on the topic of 'The Importance of Water' for a 5 year old child? Please ensure you output a word document. You must include the file you generate in the annotation of the output text using a markdown url link in this format as an example: sandbox:/mnt/data/int100.txt" }]
            }],
            reasoning={"effort": "medium", "summary": "auto"},
            text={"format": {"type": "text"}},
            include=["code_interpreter_call.outputs"],
            max_output_tokens=32000
        )
        
         # Print the text response
        print("Text response:")
        async for chunk in response_generator:
            print(chunk, end="", flush=True)
            
    except Exception as e:
        print(f"Error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the async function with await
await run_code_interpreter()
 

INFO:llm_master.openai_provider:Successfully initialized OpenAI provider


Text response:
Instantiating provider: openai_provider with class OpenAIProvider


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


```py
from docx import Document

# Create a new Word document
doc = Document()

# Add title and basic information
doc.add_heading("Lesson Plan: The Importance of Water", level=1)
doc.add_paragraph("Age Group: 5 year olds")
doc.add_paragraph("Duration: 30 minutes")

# Add objectives
doc.add_heading("Objectives", level=2)
objectives = [
    "Understand why water is important for life",
    "Identify sources of water",
    "Practice good water use habits"
]
for obj in objectives:
    doc.add_paragraph(f"• {obj}")

# Add materials
doc.add_heading("Materials", level=2)
materials = [
    "A glass of water",
    "Pictures of water uses (plants, animals, drinking)",
    "Coloring sheets with water drops",
    "Crayons"
]
for item in materials:
    doc.add_paragraph(f"• {item}")

# Add lesson outline
doc.add_heading("Lesson Outline", level=2)

# Introduction
doc.add_heading("1. Introduction (5 min)", level=3)
intro_steps = [
    "Greet the children.",
    "Show a glass of water; ask what it is.

In [ ]:
{"type":"response.output_item.done","sequence_number":877,"output_index":2,"item":{"id":"msg_6838c205884c8191a9ef3e5e8c5378170a1145d46963df26","type":"message","status":"completed","content":[{"type":"output_text","annotations":[{"type":"container_file_citation","container_id":"cntr_6838c1f76ff88191b27d1dcd064592550c88636f8949b334","end_index":219,"file_id":"cfile_6838c20627bc81919fbf3bae04fa20e2","filename":"lesson_plan_importance_of_water.docx","start_index":165}],"text":"I’ve created the lesson plan document for \"The Importance of Water\" suitable for a 5-year-old. You can download it here:\n\n[Download the lesson plan (Word document)](sandbox:/mnt/data/lesson_plan_importance_of_water.docx)"}],"role":"assistant"}}

In [ ]:


async def run_stream():
    messages = [
        {
            "role": "user", 
            "content": "what do these images show? ",
            "images":[
                "https://uflo-chat-attachments.s3.us-west-1.amazonaws.com/218/215c88fa-952f-4a21-80d7-2af01ef1c191/ca106aaf-bfa3-45b9-bf70-eb612bbe27d0.png",
                "https://uflo-chat-attachments.s3.us-west-1.amazonaws.com/218/215c88fa-952f-4a21-80d7-2af01ef1c191/00b5fe2d-ff07-43e1-8f2e-ab73906a30b7.png",
                "https://uflo-chat-attachments.s3.us-west-1.amazonaws.com/218/215c88fa-952f-4a21-80d7-2af01ef1c191/2cc690db-8cc4-4737-8e97-ac96b4256793.png"
            ]
        },
    ]

    try:
        response_generator = await llm.query(
            # model_name="o3-mini",
            reasoning_effort="none",
            model_name="gemini-2.5-flash-preview-04-17",
            # model_name="sonar-pro",
            # model_name="responses-o4-mini",
            # reasoning={"effort": "high", "summary": "auto"},
            # max_output_tokens=4096,
            # max_tokens=4096,
            # model_name="accounts/fireworks/models/deepseek-r1",
            messages=messages,
            stream=True,
            # fallback_provider="openai",
            # fallback_model="gpt-4o",
            moderation=False,
            # extra_body={
            #     "return_images": True,
            #     "web_search_options": {
            #         "search_context_size": "low"
            #     }
            # },
        )
        
        async for chunk in response_generator:
            print(chunk, end="", flush=True)
            
    except Exception as e:
        print(f"Error: {str(e)}")

# This line is key in Jupyter
await run_stream()  # Don't use asyncio.run() here

In [ ]:


async def run_query():
    messages = [
        {
            "role": "user", 
            "content": "Explain quantum computing in simple terms",
        },
    ]

    try:
        # With stream=False, you get a LLMResponse object directly
        response = await llm.query(
            # model_name="accounts/fireworks/models/deepseek-r1",
            model_name="gemini-2.0-flash",
            # reasoning_effort="low",
            messages=messages,
            stream=False,  # Set to False for non-streaming
            # temperature=0.5,
            fallback_provider="openai",
            fallback_model="gpt-4o",
            moderation=False
        )
        
        # Print the full response content
        print(response.content)
        
        # You can also access other metadata
        print("\n--- Response Metadata ---")
        print(f"Model: {response.model_name}")
        print(f"Input tokens: {response.usage.input_tokens}")
        print(f"Output tokens: {response.usage.output_tokens}")
        print(f"Cost: ${response.cost:.6f}")
        print(f"Latency: {response.latency:.2f} seconds")
            
    except Exception as e:
        print(f"Error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the async function with await
await run_query()

In [ ]:

async def test_perplexity():
    # Sample messages
    messages = [
        {
            "role": "system",
            "content": "You are an artificial intelligence assistant and you need to engage in a helpful, detailed, polite conversation with a user. Answer as concisely as possible."
        },
        {   
            "role": "user",
            "content": "How many stars are in the universe?"
        },
    ]
    
    # Test non-streaming query
    print("Testing non-streaming Perplexity API with citations")
    response = await llm.query(
        model_name="sonar",
        messages=messages,
        stream=False
    )
    
    print(f"Response content: {response.content}")
    # print(f"Citations: {response.citations}")
    
    # Test streaming query
    print("\nTesting streaming Perplexity API with citations")
    stream_generator = await llm.query(
        model_name="sonar",
        messages=messages,
        stream=True
    )
    
    full_response = ""
    citations_found = False
    
    async for chunk in stream_generator:
        if chunk.startswith("\n<citations>"):
            print(f"Found citations in stream: {chunk}")
            citations_found = True
        else:
            full_response += chunk
            print(f"Received chunk: {chunk}")
    
    print(f"\nFull response: {full_response[:100]}...")
    
    # After streaming is complete, check if provider has citations
    provider = llm._get_provider("sonar")
    if hasattr(provider, 'last_citations') and provider.last_citations:
        print(f"Citations from provider.last_citations: {provider.last_citations}")
        citations_found = True
    
    if not citations_found:
        print("No citations found in streaming response")

await test_perplexity()


In [ ]:
# Define the prompt
prompt = "A cat on its back legs running like a human is holding a big silver fish with its arms. The cat is running away from the shop owner and has a panicked look on his face. The scene is situated in a crowded market."

try:
    # Generate the image
    response = await llm.query(
        model_name="recraftv3",
        messages=[{"role": "user", "content": prompt}],
        style="digital_illustration"
    )
    
    # Print the result (which is the image URL)
    print(f"Image generation successful! URL: {response.content}")
    
except Exception as e:
    print(f"Error generating image: {str(e)}")

In [ ]:

async def run_query():
    messages = [
        {
            "role": "user", 
            "content": "Explain quantum computing in simple terms",
        },
    ]

    try:
        # With stream=False, you get a LLMResponse object directly
        response = await llm.query(
            model_name="gpt-4o-mini-audio-preview",
            messages=messages,
            stream=False,
            modality=["text", "audio"],
            audio={"voice": "ash", "format": "wav"}
        )
        
         # Print the text response
        print("Text response:")
        print(response.content)
        
        # Save the audio to a file if available
        if response.audio_data:
            wav_bytes = base64.b64decode(response.audio_data)
            output_file = "dog_response.wav"
            with open(output_file, "wb") as f:
                f.write(wav_bytes)
            print(f"\nAudio saved to '{output_file}'")
        else:
            print("\nNo audio data received in the response")
            
    except Exception as e:
        print(f"Error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the async function with await
await run_query()

In [ ]:

async def run_query():
    messages = [
        {
            "role": "user", 
            "content": f"Narrate the text: "
        },
    ]

    try:
        # With stream=False, you get a LLMResponse object directly
        response = await llm.query(
            model_name="gpt-4o-mini-audio-preview",
            messages=messages,
            stream=False,
            modality=["text", "audio"],
            audio={"voice": "ash", "format": "wav"}
        )
        
         # Print the text response
        print("Text response:")
        print(response.content)
        
        # Save the audio to a file if available
        if response.audio_data:
            wav_bytes = base64.b64decode(response.audio_data)
            output_file = "dog_response.wav"
            with open(output_file, "wb") as f:
                f.write(wav_bytes)
            print(f"\nAudio saved to '{output_file}'")
        else:
            print("\nNo audio data received in the response")
            
    except Exception as e:
        print(f"Error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the async function with await
await run_query()

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="o3",
    tools=[{
        "type": "web_search_preview",
        "search_context_size": "low",
    }],
    input="What movie won best picture in 2025?",
)

print(response.output_text)